In [ ]:
import json
import pandas
import time
import os
import csv
from functools import reduce

In [ ]:
inputDir = '..//scraped_json//'
outputDir = '..//summarized_json//'
dataset = 'sports247'

## Load the id config
idConfig = json.loads(open('..//config//idConfig.json', "r").read())

## Load the source file dict
sourceFiles = json.loads(open('..//config//sourceFiles.json', "r").read())

In [ ]:
## If there are multiple files for any given dataset, then this function will combine all of those
## records into a single dicti
def mergeSourceFiles (source, outputDir, sourceFiles):
    recordList = []
    for file in sourceFiles[source]:  
        file = json.loads(open(outputDir + file, "r", encoding="utf-8").read())
        for record in file:
            recordList.append(record)
    return recordList

In [ ]:
sports247Data = mergeSourceFiles(dataset, inputDir, sourceFiles)

In [ ]:
## Clean up dirty names
def mungeID(playerString):
    return ''.join(e for e in playerString if e.isalnum()).lower().replace("jr.", "").replace("st.", "state")  

In [ ]:
def createNewID (fieldList, thisDict, fieldAgg):
    finalID= ''
    for i in thisDict:
        i['displayName'] = i['playerName']
        for idx, val in enumerate(fieldList):
            if (type(i[val]) is list):
                i[val]= mungeID(i[val][0])
                if (len(fieldList) -1 == idx):
                    finalID += str(i[val]).strip('[]').strip("''")
            elif (type(val) is not list):
                i[val] = mungeID(i[val])
                if (len(fieldList) - 1 == idx):
                    finalID += i[val]
                else:
                    finalID = i[val] + fieldAgg
        i['ID'] = finalID
        finalID=''

In [ ]:
createNewID(idConfig[dataset], sports247Data, '_')
df = pandas.DataFrame(sports247Data)
df.head()

,school,year,playerName,highSchool,city,state,position,height,weight,compRating,...,nationalRank,positionRank,stateRank,247Rating,247Stars,247positionRank,247stateRank,displayName,ID,247nationalRank
0,nebraska,2012,paulthurston,Arvada West,Arvada,CO,OT,77,275,0.9407,...,152,16,2,92,4.0,26,2,Paul Thurston,paulthurston_nebraska,NaN
1,nebraska,2012,gregmcmullen,Archbishop Hoban,Akron,OH,SDE,77,255,0.9099,...,269,14,18,95,4.0,6,8,Greg McMullen,gregmcmullen_nebraska,139
2,nebraska,2012,jordanwesterkamp,Montini Catholic,Lombard,IL,WR,73,190,0.9134,...,254,36,6,90,4.0,46,7,Jordan Westerkamp,jordanwesterkamp_nebraska,NaN
3,nebraska,2012,imanicross,North Hall,Gainesville,GA,RB,73,222,0.9093,...,273,15,23,87,3.0,27,40,Imani Cross,imanicross_nebraska,NaN
4,nebraska,2012,tommyarmstrong,Steele,Cibolo,TX,DUAL,73,200,0.9060,...,286,6,42,86,3.0,15,110,Tommy Armstrong,tommyarmstrong_nebraska,NaN


In [ ]:
##Since these are the main fields, I'm going to clean them up a bit.
final247 = []
for record in sports247Data:
    finalRecord = {}
    finalRecord['ID'] = record['ID']
    finalRecord['playerName'] = record['displayName']
    finalRecord['year'] = record['year']
    finalRecord['college'] = record['school']
    finalRecord['highSchool'] = record['highSchool']
    finalRecord['homeCity'] = record['city']
    finalRecord['homeState'] = record['state']
    finalRecord['position'] = record['position']
    finalRecord['height'] = record['height']
    finalRecord['weight'] = record['weight']
    finalRecord['comp_stars'] = record['compStars']
    finalRecord['comp_rating'] = record['compRating']
    finalRecord['comp_natlRank'] = record['nationalRank']
    finalRecord['comp_posRank'] = record['positionRank']
    finalRecord['comp_stateRank'] = record['stateRank']
    final247.append(finalRecord)

df= pandas.DataFrame(final247)
df.head()

,ID,playerName,year,college,highSchool,homeCity,homeState,position,height,weight,comp_stars,comp_rating,comp_natlRank,comp_posRank,comp_stateRank
0,paulthurston_nebraska,Paul Thurston,2012,nebraska,Arvada West,Arvada,CO,OT,77,275,4,0.9407,152,16,2
1,gregmcmullen_nebraska,Greg McMullen,2012,nebraska,Archbishop Hoban,Akron,OH,SDE,77,255,4,0.9099,269,14,18
2,jordanwesterkamp_nebraska,Jordan Westerkamp,2012,nebraska,Montini Catholic,Lombard,IL,WR,73,190,4,0.9134,254,36,6
3,imanicross_nebraska,Imani Cross,2012,nebraska,North Hall,Gainesville,GA,RB,73,222,4,0.9093,273,15,23
4,tommyarmstrong_nebraska,Tommy Armstrong,2012,nebraska,Steele,Cibolo,TX,DUAL,73,200,4,0.9060,286,6,42


In [ ]:
with open(outputDir + dataset + "data.json", "w", encoding="utf-8") as write_file:
                write_file.write(json.dumps(final247))